In [1]:
# Dependencies
import requests
from config import api_key
import pandas as pd
import json
import os
from sqlalchemy import create_engine
import numpy as np
from pprint import pprint 
from sqlalchemy.orm import Session

In [2]:
wiki_data = pd.read_csv("Resources/wiki_movie_plots.csv")
wiki_data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Unnamed: 8
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",NaN
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",NaN
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",NaN
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,NaN
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,NaN


In [3]:
imdb_data = pd.read_csv("Resources/imdb_top_1000.csv")
imdb_data.head()


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [ ]:
#keeping the IMDB table and do a merge
#extract the data
#clean the data


In [4]:
#cleaning the wiki_data to removed the "s messy and his clothes are torn. He looks ......" text from the year released column
wiki_data = wiki_data[wiki_data["Release Year"].str.contains("s messy and his clothes are torn. He looks at his army who watch in shock.")==False]

#cleaning the wiki_data to removed the "Rama is praying for safety but no help is...." text from the year released column
wiki_data = wiki_data[wiki_data["Release Year"].str.contains("Rama is praying for safety but no help is given to him. Rama is presumed dead but Sita prays and the chakra grows bigger. With great difficulty the hurt Rama crawls towards the distant chakra. He is moving very slowly and screaming in pain. His wounds are getting much worse and his bones are broken. He is struggling to live. Ravana is about to kill the weak Rama. Rama is stopping and crying because he was too weak to save Sita. Everyone else knew that Rama was about to die and failed to rescue his wife after a 4-year battle.But before Ravana could kill the wounded Rama")==False]

#cleaning the wiki_data to removed the "With the battle over" text from the year released column
wiki_data = wiki_data[wiki_data["Release Year"].str.contains("With the battle over")==False]

wiki_data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Unnamed: 8
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",NaN
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",NaN
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",NaN
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,NaN
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,NaN


In [5]:
#Change the Released Year datatype in the wiki_data_clean from object to Int64 datatype so that we can merge the two datasets
wiki_data['Release Year'] = pd.to_numeric(wiki_data['Release Year'])
wiki_data.dtypes

Release Year         int64
Title               object
Origin/Ethnicity    object
Director            object
Cast                object
Genre               object
Wiki Page           object
Plot                object
Unnamed: 8          object
dtype: object

In [6]:
#cleaning the imdb_data to removed the "PG" text from the year released column
imdb_data = imdb_data[imdb_data["Released_Year"].str.contains("PG")==False]

#Change the Released_Year datatype from object to Int64 datatype so that we can merge the two datasets
imdb_data['Released_Year'] = pd.to_numeric(imdb_data['Released_Year'])

#imdb_data.dtypes
imdb_data.dtypes

Poster_Link       object
Series_Title      object
Released_Year      int64
Certificate       object
Runtime           object
Genre             object
IMDB_Rating      float64
Overview          object
Meta_score       float64
Director          object
Star1             object
Star2             object
Star3             object
Star4             object
No_of_Votes        int64
Gross             object
dtype: object

In [7]:
movies_merge_df = pd.merge(imdb_data, wiki_data, left_on = "Released_Year", right_on = "Release Year", how = "left", suffixes=('_',''))

# movies_merge_df = pd.merge(imdb_data, wiki_data, left_on = "Released_Year", right_on = "Release Year", how = "inner")

# movies_merge_df = pd.merge(imdb_data, wiki_data, how="left", on=["Released_Year", "Release Year"])
movies_merge_df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre_,IMDB_Rating,Overview,Meta_score,Director_,...,Gross,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Unnamed: 8
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,...,"28,341,469",1994.0,3 Chains o' Gold,American,Three directors,NaN,musical,https://en.wikipedia.org/wiki/3_Chains_o%27_Go...,The story begins with the assassination of Egy...,NaN
1,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,...,"28,341,469",1994.0,3 Ninjas Kick Back,American,Charles T. Kanganis,"Victor Wong, Max Elliott Slade, Sean Fox, J. E...","action, family",https://en.wikipedia.org/wiki/3_Ninjas_Kick_Back,"Rocky, Colt and Tum-Tum are experiencing the p...",NaN
2,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,...,"28,341,469",1994.0,8 Seconds,American,John G. Avildsen,"Luke Perry, Stephen Baldwin, Cynthia Gibb",biopic,https://en.wikipedia.org/wiki/8_Seconds,"While growing up in Oklahoma, young Lane Frost...",NaN
3,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,...,"28,341,469",1994.0,Above the Rim,American,Jeff Pollack,"Duane Martin, Leon, Tupac Shakur",drama,https://en.wikipedia.org/wiki/Above_the_Rim,Kyle Watson is a talented basketball player wh...,NaN
4,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,...,"28,341,469",1994.0,Abraham,American,Joseph Sargent,"Richard Harris, Barbara Hershey",drama,https://en.wikipedia.org/wiki/Abraham_(film),"Abram lives in Harran, a rich city. His wife S...",NaN


In [8]:
# #dropping all un-needed column
movies_merge_df.drop(['Poster_Link', 'Series_Title', 'Genre_', 'Certificate', 'Star1', 'Star2', 'Star3', 'Star4', 'Director_', 'Unnamed: 8', 'Wiki Page'], axis=1, inplace=True)
movies_merge_df 
#movies_merge_df.head()



,Released_Year,Runtime,IMDB_Rating,Overview,Meta_score,No_of_Votes,Gross,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Plot
0,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,3 Chains o' Gold,American,Three directors,NaN,musical,The story begins with the assassination of Egy...
1,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,3 Ninjas Kick Back,American,Charles T. Kanganis,"Victor Wong, Max Elliott Slade, Sean Fox, J. E...","action, family","Rocky, Colt and Tum-Tum are experiencing the p..."
2,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,8 Seconds,American,John G. Avildsen,"Luke Perry, Stephen Baldwin, Cynthia Gibb",biopic,"While growing up in Oklahoma, young Lane Frost..."
3,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,Above the Rim,American,Jeff Pollack,"Duane Martin, Leon, Tupac Shakur",drama,Kyle Watson is a talented basketball player wh...
4,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,Abraham,American,Joseph Sargent,"Richard Harris, Barbara Hershey",drama,"Abram lives in Harran, a rich city. His wife S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479730,1935,86 min,7.6,A man in London tries to help a counter-espion...,93.0,51853,NaN,1935.0,Dr. Madhurika a.k.a. Modern Wife,Bollywood,Sarvottam Badami,"Motilal, Sabita Devi, Padma Shaligram, Bhudo A...",social family drama,"Dr. Madhurika (Sabita Devi) is a ""modern"" youn..."
479731,1935,86 min,7.6,A man in London tries to help a counter-espion...,93.0,51853,NaN,1935.0,Hind Kesari,Bollywood,Homi Wadia,"Husn Bano, Sardar Mansur, Dilawar, Gulshan, Ja...",action,King Mansingh (Tarapore) falls prey to the mac...
479732,1935,86 min,7.6,A man in London tries to help a counter-espion...,93.0,51853,NaN,1935.0,Hunterwali a.k.a. The Lady With The Whip,Bollywood,Homi Wadia,"Fearless Nadia, John Cawas, Boman Shroff, Shar...",action,"The story begins on a stormy night, with a pro..."
479733,1935,86 min,7.6,A man in London tries to help a counter-espion...,93.0,51853,NaN,1935.0,Karwan-E-Hayat,Bollywood,"Premankur Atorthy, Hemchandra Chunder","K. L. Saigal, Molina, Pahari Sanyal, Rajkumari...",costume action adventure,"Pervez K. L. Saigal, heir to the throne of Kas..."


In [9]:
#dropping all duplicates rows

movies_merge_df.drop_duplicates(keep='first', inplace=True, ignore_index=False)
movies_merge_df.head()

,Released_Year,Runtime,IMDB_Rating,Overview,Meta_score,No_of_Votes,Gross,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Plot
0,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,3 Chains o' Gold,American,Three directors,NaN,musical,The story begins with the assassination of Egy...
1,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,3 Ninjas Kick Back,American,Charles T. Kanganis,"Victor Wong, Max Elliott Slade, Sean Fox, J. E...","action, family","Rocky, Colt and Tum-Tum are experiencing the p..."
2,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,8 Seconds,American,John G. Avildsen,"Luke Perry, Stephen Baldwin, Cynthia Gibb",biopic,"While growing up in Oklahoma, young Lane Frost..."
3,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,Above the Rim,American,Jeff Pollack,"Duane Martin, Leon, Tupac Shakur",drama,Kyle Watson is a talented basketball player wh...
4,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,Abraham,American,Joseph Sargent,"Richard Harris, Barbara Hershey",drama,"Abram lives in Harran, a rich city. His wife S..."


In [10]:
#dropping all rows with NAN fields
movies_merge_df.dropna(axis=0, how= "any", inplace=True)
movies_merge_df.head()

,Released_Year,Runtime,IMDB_Rating,Overview,Meta_score,No_of_Votes,Gross,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Plot
1,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,3 Ninjas Kick Back,American,Charles T. Kanganis,"Victor Wong, Max Elliott Slade, Sean Fox, J. E...","action, family","Rocky, Colt and Tum-Tum are experiencing the p..."
2,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,8 Seconds,American,John G. Avildsen,"Luke Perry, Stephen Baldwin, Cynthia Gibb",biopic,"While growing up in Oklahoma, young Lane Frost..."
3,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,Above the Rim,American,Jeff Pollack,"Duane Martin, Leon, Tupac Shakur",drama,Kyle Watson is a talented basketball player wh...
4,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,Abraham,American,Joseph Sargent,"Richard Harris, Barbara Hershey",drama,"Abram lives in Harran, a rich city. His wife S..."
5,1994,142 min,9.3,Two imprisoned men bond over a number of years...,80.0,2343110,"28,341,469",1994.0,Ace Ventura: Pet Detective,American,Tom Shadyac,"Jim Carrey, Courteney Cox, Sean Young, Tone Lo...",comedy,Ace Ventura is an eccentric Miami-based privat...


In [11]:
#create a non-sql database
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [ ]:
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

In [13]:
# Create Surfer and Board classes
#Series_Title	Released_Year	Runtime	Genre_	IMDB_Rating	Overview	Meta_score	No_of_Votes	Gross	Release Year	Title	Origin/Ethnicity	Director	Cast	Genre	Plot
# ----------------------------------
class Surfer(Base):
    __tablename__ = 'movies_info'
    id = Column(Integer, primary_key=True)
    Released_Year = Column(Integer)
    Runtime = Column(String(10))
    IMDB_Rating = Column(String(5))
    Overview = Column(String(255))
    Meta_score = Column(Integer)
    No_of_Votes = Column(Integer)
    Gross = Column(Integer)
    Title = Column(String(50))
    Origin_Ethnicity = Column(String(50))
    Director = Column(String(20))
    Cast = Column(String(100))
    Genre = Column(String(50))
    Plot = Column(String(100))

In [15]:
# Establish Connection
engine = create_engine('sqlite://..Resources/movies_info.sqlite')
Base.metadata.create_all(engine)

In [19]:
# Create both the Surfer and movies_info table within the database
Base.metadata.create_all(conn)

In [ ]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
# Query the database and collect all of the surfers in the Surfer table
movies_list = session.query(movies_info)
for movie in movies_list:
    print(movie.Title)
    print(movie.Director)
    print(movie.Plot)